# Гладышев ВВ Курсовая работа

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pylab import rcParams
from scipy import stats
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

## Подготовка данных

In [2]:
data = pd.read_csv("train.csv", sep=",")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [4]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [5]:
data.corr()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
Id,1.000000,0.012973,-0.005847,-0.010071,0.018449,0.019880,0.001348,-0.008376,0.005004,0.018097,-0.000772,-0.002033,-0.009358,-0.003879,0.001502,-0.008718,0.009880
DistrictId,0.012973,1.000000,0.071432,-0.026613,-0.019910,0.040358,-0.120373,-0.149051,0.013430,0.065294,0.246463,0.167479,0.136095,0.304335,0.306147,0.174214,0.265100
Rooms,-0.005847,0.071432,1.000000,0.662893,0.131336,0.005123,-0.000665,-0.029302,-0.010612,-0.032347,0.075980,0.071335,0.012811,0.042857,0.063557,0.053618,0.550291
Square,-0.010071,-0.026613,0.662893,1.000000,0.196129,0.008320,0.114791,0.081505,-0.009032,-0.064479,-0.070690,-0.043120,0.035241,-0.039748,-0.022960,0.021357,0.520075
LifeSquare,0.018449,-0.019910,0.131336,0.196129,1.000000,-0.001195,0.024559,0.027442,-0.002210,-0.023629,-0.048363,-0.039283,0.012763,-0.026867,-0.024762,-0.007569,0.081292
KitchenSquare,0.019880,0.040358,0.005123,0.008320,-0.001195,1.000000,-0.011397,0.000780,0.000958,-0.005622,0.043379,0.037805,-0.015140,0.009472,0.043170,0.010216,0.028864
Floor,0.001348,-0.120373,-0.000665,0.114791,0.024559,-0.011397,1.000000,0.418986,0.000928,-0.016133,-0.044914,-0.016560,-0.002237,-0.138294,-0.065537,0.024264,0.128715
HouseFloor,-0.008376,-0.149051,-0.029302,0.081505,0.027442,0.000780,0.418986,1.000000,-0.000864,-0.004362,-0.020801,0.007194,-0.008137,-0.143973,-0.068728,0.026279,0.088280
HouseYear,0.005004,0.013430,-0.010612,-0.009032,-0.002210,0.000958,0.000928,-0.000864,1.000000,0.001465,0.003026,0.001970,0.000819,-0.011969,0.011245,0.003681,0.004305
Ecology_1,0.018097,0.065294,-0.032347,-0.064479,-0.023629,-0.005622,-0.016133,-0.004362,0.001465,1.000000,0.026464,0.009264,-0.124068,-0.043547,0.030873,-0.076749,-0.058381


In [6]:
data['DistrictId'].nunique()

205

In [7]:
data.groupby('DistrictId').count()

,Id,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
DistrictId,,,,,,,,,,,,,,,,,,,
0,48,48,48,45,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
1,652,652,652,263,652,652,652,652,652,652,652,652,652,652,57,652,652,652,652
2,78,78,78,78,78,78,78,78,78,78,78,78,78,78,59,78,78,78,78
3,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82
4,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
5,34,34,34,33,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
6,511,511,511,232,511,511,511,511,511,511,511,511,511,511,511,511,511,511,511
7,92,92,92,91,92,92,92,92,92,92,92,92,92,92,89,92,92,92,92
8,142,142,142,142,142,142,142,142,142,142,142,142,142,142,118,142,142,142,142


In [8]:
data.groupby('DistrictId')['DistrictId'].count().sort_values()

DistrictId
209      1
207      1
205      1
117      1
199      1
196      1
174      1
202      2
208      2
195      2
158      2
172      2
186      2
181      3
185      3
175      3
201      3
193      3
112      4
136      4
182      4
50       4
180      4
179      4
194      5
188      5
133      5
166      5
115      5
65       5
      ... 
3       82
41      87
94      89
29      89
48      90
7       92
101     97
99      97
13      97
47      97
44     104
57     107
34     111
74     114
45     116
61     118
46     119
11     129
8      142
52     143
21     165
53     174
58     179
30     228
62     247
9      294
6      511
23     565
1      652
27     851
Name: DistrictId, Length: 205, dtype: int64

In [9]:
data = data.drop('Healthcare_1', axis=1)

In [10]:
data.loc[(~data['HouseYear'].between(1915, 2020)), 'HouseYear'].sort_values()

9317        1910
2656        1912
5111        1912
910         1914
4189        4968
1497    20052011
Name: HouseYear, dtype: int64

In [11]:
data.loc[4189, 'HouseYear'] = 1968
data.loc[1497, 'HouseYear'] = 2011

In [12]:
data.loc[data['Rooms']>5]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,5,15,B,317265.323792
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,B,B,1,264,0,0,1,B,78364.616704
2170,14003,99,6.0,59.414334,38.702244,6.0,7,9.0,1969,0.033494,B,B,66,10573,1,3,8,B,229661.964416
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994,0.161532,B,B,25,5648,1,2,4,B,172329.270863


In [13]:
data.loc[377, 'Rooms'] = 3
data.loc[1454, 'Rooms'] = 2
data.loc[2170, 'Rooms'] = 3
data.loc[8849, 'Rooms'] = 3

In [14]:
data.loc[data['Rooms']==0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,0,0,B,268394.744389
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,0,1,B,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1,0,B,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,0,1,B,219281.918007


In [15]:
data.drop(data.loc[data['Rooms']==0].index, axis=0, inplace=True)

In [16]:
data.loc[data['Rooms']==5, 'Square']

1422     95.617533
1503    108.057398
1982    275.645284
3268    121.577423
3271     42.003293
4431     39.906082
4487    183.382273
4762    114.313752
5361    118.871952
5466    112.135525
6623    111.045799
6988    124.933289
8511    198.930182
8659    119.008007
8726     90.762276
8925     85.313605
9432    147.709302
9699    117.855969
Name: Square, dtype: float64

In [17]:
data.drop([3271, 4431], axis=0, inplace=True)

In [18]:
data.loc[6977]

Id                     11602
DistrictId                30
Rooms                      2
Square               641.065
LifeSquare           638.163
KitchenSquare             10
Floor                     20
HouseFloor                19
HouseYear               2019
Ecology_1        7.81528e-05
Ecology_2                  B
Ecology_3                  B
Social_1                  22
Social_2                6398
Social_3                 141
Helthcare_2                3
Shops_1                   23
Shops_2                    B
Price                 133530
Name: 6977, dtype: object

In [19]:
data.loc[6977, 'Square'] = 64.1065
data.loc[6977, 'LifeSquare'] = 63.8163
data.loc[4262, 'Square'] = 60.4706
data.loc[4262, 'LifeSquare'] = 60.3706
data.loc[4690, 'Square'] = 40.9425
data.loc[4690, 'LifeSquare'] = 41.064
data.loc[4328, 'LifeSquare'] = 74.8059
data.loc[1064, 'KitchenSquare'] = data.loc[1064, 'Square'] - data.loc[1064, 'LifeSquare']
data.loc[8584, 'KitchenSquare'] = data.loc[8584, 'Square'] - data.loc[8584, 'LifeSquare']

In [20]:
data.loc[data['Square']<15]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
212,1748,88,2.0,5.497061,67.628717,1.0,24,22.0,1977,0.127376,B,B,43,8429,3,3,9,B,412511.088764
1316,11526,27,1.0,4.633498,1.969969,1.0,18,1.0,1977,0.011654,B,B,4,915,0,0,0,B,107604.269441
1608,10202,6,1.0,2.596351,4.604943,1.0,3,25.0,2014,0.243205,B,B,5,1564,0,0,0,B,137597.601458
3280,10527,27,1.0,4.380726,40.805837,1.0,10,17.0,2013,0.211401,B,B,9,1892,0,0,1,B,97560.720383
3413,9487,5,1.0,5.129222,5.549458,1.0,1,1.0,1977,0.150818,B,B,16,3433,4,4,5,B,369472.403061
4739,12676,81,3.0,13.784865,15.988889,7.0,4,5.0,1960,0.319809,B,B,25,4756,16,5,8,B,78388.806186
4900,4504,27,3.0,4.390331,5.610772,1.0,8,19.0,2016,0.211401,B,B,9,1892,0,0,1,B,161379.067034
6392,14786,1,1.0,1.136859,4.525736,1.0,3,1.0,1977,0.007122,B,B,1,264,0,0,1,B,181434.825589
8030,13265,1,3.0,4.823679,79.767964,0.0,6,17.0,1977,0.007122,B,B,1,264,0,0,1,B,237716.681261
8283,15744,34,1.0,1.988943,2.642219,1.0,21,4.0,1977,0.069753,B,B,53,13670,4,1,11,B,458378.777006


In [21]:
#data = data.drop(data.loc[data['Square']<15].index, axis=0)

In [22]:
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [23]:
data.loc[data['LifeSquare']>250]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
590,14990,23,2.0,48.449873,263.542020,5.0,6,5.0,1972,0.075779,B,B,6,1437,3,0,2,B,141780.231857
8437,15886,85,3.0,78.059331,461.463614,10.0,12,16.0,1998,0.037178,B,B,52,11217,1,1,7,B,394253.299978


In [24]:
data = data.drop(data.loc[data['LifeSquare']>250].index, axis=0)

## Анализ данных

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9988 entries, 0 to 9999
Data columns (total 19 columns):
Id               9988 non-null int64
DistrictId       9988 non-null int64
Rooms            9988 non-null float64
Square           9988 non-null float64
LifeSquare       9988 non-null float64
KitchenSquare    9988 non-null float64
Floor            9988 non-null int64
HouseFloor       9988 non-null float64
HouseYear        9988 non-null int64
Ecology_1        9988 non-null float64
Ecology_2        9988 non-null object
Ecology_3        9988 non-null object
Social_1         9988 non-null int64
Social_2         9988 non-null int64
Social_3         9988 non-null int64
Helthcare_2      9988 non-null int64
Shops_1          9988 non-null int64
Shops_2          9988 non-null object
Price            9988 non-null float64
dtypes: float64(7), int64(9), object(3)
memory usage: 1.5+ MB


In [26]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000,9988.000000
mean,8384.493893,50.416800,1.887865,56.148047,36.043762,5.880822,8.528734,12.617441,1984.862835,0.118861,24.695234,5353.905086,8.048058,1.320485,4.233981,214150.001089
std,4858.847431,43.603289,0.810491,19.063804,15.708483,5.174706,5.241698,6.772337,18.414821,0.119064,17.532291,4007.149896,23.844786,1.493926,4.807584,92883.382530
min,0.000000,0.000000,1.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1910.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4170.750000,20.000000,1.000000,41.773051,25.506959,1.000000,4.000000,9.000000,1974.000000,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153891.653482
50%,8396.500000,36.000000,2.000000,52.513310,36.109119,6.000000,7.000000,13.000000,1977.000000,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192255.017896
75%,12592.500000,75.000000,2.000000,65.885863,41.421759,9.000000,12.000000,17.000000,2001.000000,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.000000,209.000000,5.000000,275.645284,233.949309,123.000000,42.000000,117.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


In [27]:
data.corr()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
Id,1.000000,0.012756,-0.008326,-0.006566,-0.005651,0.010007,0.001310,-0.009025,0.003432,0.018349,-0.001010,-0.002292,-0.009443,0.001534,-0.008991,0.009418
DistrictId,0.012756,1.000000,0.076009,-0.026964,-0.057930,0.185338,-0.120780,-0.150024,-0.201148,0.065475,0.245972,0.166977,0.136007,0.305818,0.173771,0.265049
Rooms,-0.008326,0.076009,1.000000,0.764829,0.542694,0.110869,-0.005487,-0.032972,-0.043598,-0.030215,0.077306,0.072125,0.014283,0.064251,0.054120,0.572971
Square,-0.006566,-0.026964,0.764829,1.000000,0.667232,0.117215,0.118499,0.088545,0.189152,-0.072215,-0.077133,-0.047328,0.023411,-0.031218,0.011961,0.579342
LifeSquare,-0.005651,-0.057930,0.542694,0.667232,1.000000,0.040796,0.090224,0.081850,0.175853,-0.058823,-0.126884,-0.099639,0.041282,-0.062981,0.000113,0.364517
KitchenSquare,0.010007,0.185338,0.110869,0.117215,0.040796,1.000000,-0.011600,0.134619,0.167661,0.020227,0.277729,0.254615,-0.059653,0.153458,0.075925,0.247803
Floor,0.001310,-0.120780,-0.005487,0.118499,0.090224,-0.011600,1.000000,0.419177,0.284596,-0.016198,-0.045661,-0.017190,-0.002373,-0.066050,0.023914,0.128739
HouseFloor,-0.009025,-0.150024,-0.032972,0.088545,0.081850,0.134619,0.419177,1.000000,0.393924,-0.004274,-0.021776,0.006379,-0.008534,-0.069777,0.025320,0.088053
HouseYear,0.003432,-0.201148,-0.043598,0.189152,0.175853,0.167661,0.284596,0.393924,1.000000,-0.066534,-0.048135,0.013498,-0.049345,-0.116358,-0.001996,0.036101
Ecology_1,0.018349,0.065475,-0.030215,-0.072215,-0.058823,0.020227,-0.016198,-0.004274,-0.066534,1.000000,0.026661,0.009442,-0.124114,0.030937,-0.076734,-0.058458


In [28]:
data.Ecology_2.unique()

array(['B', 'A'], dtype=object)

In [29]:
data.Ecology_3.unique()

array(['B', 'A'], dtype=object)

In [30]:
data.Shops_2.unique()

array(['B', 'A'], dtype=object)

In [31]:
data.DistrictId.nunique()

205

In [32]:
data.groupby(['Ecology_2', 'Ecology_3', 'Shops_2'])['Price'].mean().sort_values()

Ecology_2  Ecology_3  Shops_2
A          B          A          159386.651546
B          B          A          192908.557092
A          B          B          198833.005156
B          B          B          215431.282150
           A          B          243179.560704
Name: Price, dtype: float64

In [33]:
data.groupby(['DistrictId', 'Ecology_2', 'Ecology_3', 'Shops_2'])['Price'].mean().sort_values()

DistrictId  Ecology_2  Ecology_3  Shops_2
23          B          B          B          126787.814437
158         B          B          B          129080.871245
23          B          B          A          134143.042318
209         B          B          B          140048.802277
27          B          B          B          146061.321223
85          B          B          A          153181.372880
57          A          B          A          158067.834043
3           B          A          B          159292.272505
58          B          B          B          162024.129640
71          A          B          A          162683.695305
6           B          B          B          166035.502703
102         B          B          B          166626.871465
30          B          B          B          170530.981705
62          B          B          A          171168.223258
93          B          B          B          171399.422402
52          B          B          B          171615.244096
126         B 

In [34]:
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)

#data=pd.get_dummies(data)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9988 entries, 0 to 9999
Data columns (total 19 columns):
Id               9988 non-null int64
DistrictId       9988 non-null int64
Rooms            9988 non-null float64
Square           9988 non-null float64
LifeSquare       9988 non-null float64
KitchenSquare    9988 non-null float64
Floor            9988 non-null int64
HouseFloor       9988 non-null float64
HouseYear        9988 non-null int64
Ecology_1        9988 non-null float64
Ecology_2        9988 non-null int64
Ecology_3        9988 non-null int64
Social_1         9988 non-null int64
Social_2         9988 non-null int64
Social_3         9988 non-null int64
Helthcare_2      9988 non-null int64
Shops_1          9988 non-null int64
Shops_2          9988 non-null int64
Price            9988 non-null float64
dtypes: float64(7), int64(12)
memory usage: 1.5 MB


In [36]:
rcParams["figure.figsize"] = 9, 9
#plt.scatter(data['Square'], data['Price'])
#plt.show()

In [37]:
#fig, ax = plt.subplots(figsize=(6, 3))

#_ = stats.probplot(
#    data['Square'],       # data
#    sparams=(16.117154, 275.645284),  # guesses at the distribution parameters
#    dist=stats.beta,   # the "dist" object
#    plot=ax            # where the data should be plotted
#)

In [38]:
data.loc[data['Square']>200]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1982,5548,86,5.0,275.645284,233.949309,26.0,12,37.0,2011,0.161976,1,0,31,7010,5,3,7,1,455264.882666
9910,16568,27,4.0,200.334539,201.627361,25.0,1,2.0,2013,0.041116,1,1,53,14892,4,1,4,1,528560.506016


In [39]:
data = data.drop(data.loc[data['Square']>200].index, axis=0)

In [40]:
#fig, ax = plt.subplots(figsize=(6, 3))

#_ = stats.probplot(
#    data['Price'],       # data
#    sparams=(59174.778028, 633233.466570),  # guesses at the distribution parameters
#    dist=stats.beta,   # the "dist" object
#    plot=ax            # where the data should be plotted
#)

In [41]:
data.to_csv("train_cor.csv", sep=",", index=False)

## Обработка тестовых данных

In [280]:
data_test = pd.read_csv("test.csv", sep=",")

In [281]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [282]:
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [283]:
data_test = data_test.drop('Healthcare_1', axis=1)

In [284]:
data_test.loc[(~data_test['HouseYear'].between(1915, 2020)), 'HouseYear'].sort_values()

3253    1908
2       1909
2691    1910
64      1911
1475    1912
2856    1914
Name: HouseYear, dtype: int64

In [285]:
data_test.loc[data_test['Rooms']>5]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2071,10793,23,6.0,110.750226,NaN,0.0,2,2.0,2015,0.014073,B,B,2,475,0,0,0,B
3217,4058,27,6.0,223.453689,104.113552,16.0,2,2.0,2017,0.041116,B,B,53,14892,4,1,4,B
3398,1435,111,17.0,52.866107,32.528342,8.0,15,17.0,1987,0.093443,B,B,23,4635,5,2,4,B


In [286]:
#data_test.drop(data_test.loc[data_test['Rooms']>5].index, axis=0, inplace=True)
data_test.loc[2071, 'Rooms'] = 3
data_test.loc[2071]

Id                  10793
DistrictId             23
Rooms                   3
Square             110.75
LifeSquare            NaN
KitchenSquare           0
Floor                   2
HouseFloor              2
HouseYear            2015
Ecology_1        0.014073
Ecology_2               B
Ecology_3               B
Social_1                2
Social_2              475
Social_3                0
Helthcare_2             0
Shops_1                 0
Shops_2                 B
Name: 2071, dtype: object

In [287]:
data_test.loc[3398, 'Rooms'] = 2
data_test.loc[3398]

Id                    1435
DistrictId             111
Rooms                    2
Square             52.8661
LifeSquare         32.5283
KitchenSquare            8
Floor                   15
HouseFloor              17
HouseYear             1987
Ecology_1        0.0934426
Ecology_2                B
Ecology_3                B
Social_1                23
Social_2              4635
Social_3                 5
Helthcare_2              2
Shops_1                  4
Shops_2                  B
Name: 3398, dtype: object

In [288]:
data_test.loc[data_test['Rooms']==0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2406,3343,58,0.0,116.824201,113.692424,0.0,3,3.0,1977,0.437885,B,B,23,5735,3,0,5,B
2524,10729,27,0.0,76.345154,42.820796,12.0,14,0.0,1977,0.017647,B,B,2,469,0,0,0,B


In [289]:
#data_test.drop(data_test.loc[data_test['Rooms']==0].index, axis=0, inplace=True)
data_test.loc[2406, 'Rooms'] = 3
data_test.loc[2406]

Id                   3343
DistrictId             58
Rooms                   3
Square            116.824
LifeSquare        113.692
KitchenSquare           0
Floor                   3
HouseFloor              3
HouseYear            1977
Ecology_1        0.437885
Ecology_2               B
Ecology_3               B
Social_1               23
Social_2             5735
Social_3                3
Helthcare_2             0
Shops_1                 5
Shops_2                 B
Name: 2406, dtype: object

In [290]:
data_test.loc[2524, 'Rooms'] = 2
data_test.loc[2524]

Id                   10729
DistrictId              27
Rooms                    2
Square             76.3452
LifeSquare         42.8208
KitchenSquare           12
Floor                   14
HouseFloor               0
HouseYear             1977
Ecology_1        0.0176471
Ecology_2                B
Ecology_3                B
Social_1                 2
Social_2               469
Social_3                 0
Helthcare_2              0
Shops_1                  0
Shops_2                  B
Name: 2524, dtype: object

In [291]:
data_test.loc[data_test['Square']<15]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,5,5,B
66,9011,53,1.0,1.378543,1.353573,1.0,1,1.0,1977,0.049637,B,B,34,7759,0,1,3,B
608,16401,30,1.0,2.645046,4.338755,1.0,2,1.0,1977,0.000078,B,B,22,6398,141,3,23,B
837,2138,27,1.0,5.647458,1.501582,1.0,1,1.0,1977,0.017647,B,B,2,469,0,0,0,B
1165,10120,6,1.0,5.100672,3.861780,1.0,3,1.0,1977,0.243205,B,B,5,1564,0,0,0,B
3697,15435,21,1.0,13.305531,9.495367,0.0,6,5.0,1965,0.194489,B,B,47,8004,3,3,5,B
4490,1165,27,1.0,2.372101,1.899119,1.0,2,17.0,1977,0.011654,B,B,4,915,0,0,0,B
4540,7855,6,1.0,4.967143,2.968086,1.0,3,1.0,2018,0.243205,B,B,5,1564,0,0,0,B
4713,170,6,2.0,2.900586,61.468563,1.0,18,17.0,2014,0.243205,B,B,5,1564,0,0,0,B
4902,1420,45,1.0,1.975769,2.900371,1.0,1,1.0,1977,0.195781,B,B,23,5212,6,3,2,B


In [292]:
#data_test.drop(data_test.loc[data_test['Square']<15].index, axis=0, inplace=True)

In [293]:
data_test.loc[data_test['Square']>200]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
3217,4058,27,6.0,223.453689,104.113552,16.0,2,2.0,2017,0.041116,B,B,53,14892,4,1,4,B
3909,1397,76,3.0,221.138768,118.055342,4.0,32,48.0,2011,0.000000,B,B,7,1660,39,1,5,B


In [302]:
#data_test.drop(data_test.loc[data_test['Square']>200].index, axis=0, inplace=True)
data_test.loc[3909, 'Square'] = 121.138768
data_test.loc[3909]

Id               1397.000000
DistrictId         76.000000
Rooms               3.000000
Square            121.138768
LifeSquare        118.055342
KitchenSquare       4.000000
Floor              32.000000
HouseFloor         48.000000
HouseYear        2011.000000
Ecology_1           0.000000
Ecology_2           1.000000
Ecology_3           1.000000
Social_1            7.000000
Social_2         1660.000000
Social_3           39.000000
Helthcare_2         1.000000
Shops_1             5.000000
Shops_2             1.000000
Name: 3909, dtype: float64

In [295]:
data_test['LifeSquare'] = data_test['LifeSquare'].fillna(data_test['LifeSquare'].mean())

In [296]:
data_test.loc[data_test['LifeSquare']>200]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
410,11533,94,2.0,48.713443,303.071094,6.0,5,12.0,1974,0.521867,B,B,25,6149,0,0,0,B


In [297]:
#data_test.drop(data_test.loc[data_test['LifeSquare']>200].index, axis=0, inplace=True)

In [298]:
map_AB = {
    'A': 0,
    'B': 1
}
data_test["Ecology_2"] = data_test["Ecology_2"].map(map_AB)
data_test["Ecology_3"] = data_test["Ecology_3"].map(map_AB)
data_test["Shops_2"] = data_test["Shops_2"].map(map_AB)

#data_test=pd.get_dummies(data_test)

In [299]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null int64
Ecology_3        5000 non-null int64
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null int64
dtypes: float64(6), int64(12)
memory usage: 703.2 KB


In [300]:
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,8412.595400,51.279200,1.907400,56.429500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,0.990400,0.970200,24.933800,5406.900000,8.262600,1.31940,4.242800,0.917600
std,4832.674037,44.179466,0.808304,18.972233,15.861084,9.950018,5.483228,6.789213,18.573149,0.120070,0.097518,0.170052,17.532202,4026.614773,23.863762,1.47994,4.777365,0.275001
min,1.000000,0.000000,1.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,25.850152,1.000000,4.000000,9.000000,1973.000000,0.019509,1.000000,1.000000,6.000000,1564.000000,0.000000,0.00000,1.000000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,36.158810,6.000000,7.000000,12.000000,1977.000000,0.072158,1.000000,1.000000,25.000000,5285.000000,2.000000,1.00000,3.000000,1.000000
75%,12598.250000,77.000000,2.000000,66.285129,41.769526,9.000000,12.000000,17.000000,2000.000000,0.195781,1.000000,1.000000,36.000000,7287.000000,5.000000,2.00000,6.000000,1.000000
max,16795.000000,212.000000,6.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,1.000000,1.000000,74.000000,19083.000000,141.000000,6.00000,23.000000,1.000000


In [301]:
data_test.to_csv("test_cor.csv", sep=",", index=False)

## Применение линейной регрессии

In [206]:
data = pd.read_csv("train_cor.csv", sep=",")

In [207]:
from sklearn.linear_model import LinearRegression

X = data[['Rooms', 'Square']]
y = data[['Price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [208]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.38371885646107073

In [209]:
X = data.drop(['Id', 'Price', 'Ecology_2','Ecology_3', 'Shops_2'], axis = 1)
#X = data.drop(['Id', 'Price', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B'], axis = 1)
y = data[['Price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [210]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.5197888664876753

In [211]:
X = data.drop(['LifeSquare', 'Id', 'Price', 'Ecology_2','Ecology_3', 'Shops_2'], axis = 1)
#X = data.drop(['LifeSquare', 'Id', 'Price', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B'], axis = 1)
y = data[['Price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [212]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.5199490629181358

In [213]:
X = data.drop(['Id', 'Price'], axis = 1)
y = data[['Price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [214]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.5272466035100265

In [215]:
X = data.drop(['Id', 'Price', 'LifeSquare'], axis = 1)
y = data[['Price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [216]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.5274133105471197

## Применение RandomForestRegressor

In [217]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model.fit(X_train, y_train.values[:, 0])
y_pred = model.predict(X_test)
full_set_err = r2_score(y_test, y_pred)
full_set_err

0.7131419795107474

### Пробую разделить на два массива

In [183]:
data = pd.read_csv("tr_01_cor.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)

In [184]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.2, random_state=42)

model_1 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_1.fit(X_train_1, y_train_1.values[:, 0])
y_pred_1 = model_1.predict(X_test_1)
full_set_err_1 = r2_score(y_test_1, y_pred_1)
full_set_err_1

0.682472218256634

In [161]:
data = pd.read_csv("tr_02_cor.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [162]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

model_2 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_2.fit(X_train_2, y_train_2.values[:, 0])
y_pred_2 = model_2.predict(X_test_2)
full_set_err_2 = r2_score(y_test_2, y_pred_2)
full_set_err_2

0.7837591194096716

In [163]:
y_pred_3 = model_2.predict(X_test_1)
full_set_err_3 = r2_score(y_test_1, y_pred_3)
full_set_err_3

0.4612038984433695

In [164]:
y_pred_4 = model_1.predict(X_test_2)
full_set_err_4 = r2_score(y_test_2, y_pred_4)
full_set_err_4

0.3073452890039876

In [165]:
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()

y_test_all = pd.concat([y_test_1, y_test_2], axis=0, ignore_index=True)
y_test_pred_all = pd.concat([pd.DataFrame(y_pred_1, columns=['Price']), pd.DataFrame(y_pred_2, columns=['Price'])], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))

summ r2: 0.7373758964350718


### Пробую разделить на четыре массива

In [166]:
data = pd.read_csv("d4_1_over_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [167]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.2, random_state=42)

model_1 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_1.fit(X_train_1, y_train_1.values[:, 0])
y_pred_1 = model_1.predict(X_test_1)
full_set_err_1 = r2_score(y_test_1, y_pred_1)
full_set_err_1

0.6651428154185997

In [168]:
lr = LinearRegression()
lr.fit(X_train_1, y_train_1)
y_pred_lr = lr.predict(X_test_1)
r2_score(y_test_1, y_pred_lr)

0.5475564933773323

In [172]:
data = pd.read_csv("d4_2_6276_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [173]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

model_2 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_2.fit(X_train_2, y_train_2.values[:, 0])
y_pred_2 = model_2.predict(X_test_2)
full_set_err_2 = r2_score(y_test_2, y_pred_2)
full_set_err_2

0.6625432532057188

In [174]:
data = pd.read_csv("d4_3_77_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [175]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, test_size=0.2, random_state=42)

model_3 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_3.fit(X_train_3, y_train_3.values[:, 0])
y_pred_3 = model_3.predict(X_test_3)
full_set_err_3 = r2_score(y_test_3, y_pred_3)
full_set_err_3

0.797641042605708

In [176]:
data = pd.read_csv("d4_4_1418_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [177]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, test_size=0.2, random_state=42)

model_4 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_4.fit(X_train_4, y_train_4.values[:, 0])
y_pred_4 = model_4.predict(X_test_4)
full_set_err_4 = r2_score(y_test_4, y_pred_4)
full_set_err_4

0.7765740126472536

In [180]:
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()

y_test_all = pd.concat([y_test_1, y_test_2, y_test_3, y_test_4], axis=0, ignore_index=True)
y_test_pred_all = pd.concat([pd.DataFrame(y_pred_1, columns=['Price']), pd.DataFrame(y_pred_2, columns=['Price']), pd.DataFrame(y_pred_3, columns=['Price']), pd.DataFrame(y_pred_4, columns=['Price'])], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))

summ r2: 0.7302038268391019


### Пробую разделить на три массива

In [187]:
data = pd.read_csv("d3_1_over_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [188]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.2, random_state=42)

model_1 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_1.fit(X_train_1, y_train_1.values[:, 0])
y_pred_1 = model_1.predict(X_test_1)
full_set_err_1 = r2_score(y_test_1, y_pred_1)
full_set_err_1

0.6584249461176703

In [189]:
data = pd.read_csv("d3_2_77_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [190]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

model_2 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_2.fit(X_train_2, y_train_2.values[:, 0])
y_pred_2 = model_2.predict(X_test_2)
full_set_err_2 = r2_score(y_test_2, y_pred_2)
full_set_err_2

0.797641042605708

In [191]:
data = pd.read_csv("d3_3_1418_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [192]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, test_size=0.2, random_state=42)

model_3 = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model_3.fit(X_train_3, y_train_3.values[:, 0])
y_pred_3 = model_3.predict(X_test_3)
full_set_err_3 = r2_score(y_test_3, y_pred_3)
full_set_err_3

0.7765740126472536

In [193]:
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()

y_test_all = pd.concat([y_test_1, y_test_2, y_test_3], axis=0, ignore_index=True)
y_test_pred_all = pd.concat([pd.DataFrame(y_pred_1, columns=['Price']), pd.DataFrame(y_pred_2, columns=['Price']), pd.DataFrame(y_pred_3, columns=['Price'])], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))

summ r2: 0.718230526020318


In [ ]:
y_pred_1 = model.predict(X_test_1)

full_set_err_1 = r2_score(y_test_c, y_pred_1)
full_set_err_1

### Датасет разделён на три массива. Модель для первой группы обучается с полным массивом

In [271]:
data = pd.read_csv("train_cor.csv", sep=",")

In [272]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(max_depth=16, n_estimators=1600, random_state=42)
model.fit(X_train, y_train.values[:, 0])
y_pred = model.predict(X_test)
full_set_err = r2_score(y_test, y_pred)
full_set_err

0.7283949507628684

In [273]:
data = pd.read_csv("d3_1_over_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [274]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.2, random_state=42)

y_pred_1 = model.predict(X_test_1)

full_set_err_1 = r2_score(y_test_1, y_pred_1)
full_set_err_1

0.863674370177073

In [275]:
data = pd.read_csv("d3_2_77_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [276]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

model_2 = RandomForestRegressor(max_depth=16, n_estimators=1600, random_state=42)
model_2.fit(X_train_2, y_train_2.values[:, 0])
y_pred_2 = model_2.predict(X_test_2)
full_set_err_2 = r2_score(y_test_2, y_pred_2)
full_set_err_2

0.798575365953557

In [270]:
for ind in range(1, 21):
    model_v = RandomForestRegressor(max_depth=16, n_estimators = ind * 100, random_state=42)
    model_v.fit(X_train_2, y_train_2.values[:, 0])
    y_pred_v = model_v.predict(X_test_2)
    print("n_estimators: {} r2: {}".format(ind * 100, r2_score(y_test_2, y_pred_v)))

n_estimators: 100 r2: 0.7897008687037799
n_estimators: 200 r2: 0.7945191457705448
n_estimators: 300 r2: 0.7951822067886667
n_estimators: 400 r2: 0.7958198817290738
n_estimators: 500 r2: 0.7975355716645139
n_estimators: 600 r2: 0.7974568327379635
n_estimators: 700 r2: 0.797641042605708
n_estimators: 800 r2: 0.7969878391310499
n_estimators: 900 r2: 0.797031858148521
n_estimators: 1000 r2: 0.7974398996896812
n_estimators: 1100 r2: 0.797606274058783
n_estimators: 1200 r2: 0.798146349152979
n_estimators: 1300 r2: 0.7981685314709615
n_estimators: 1400 r2: 0.7982650286042241
n_estimators: 1500 r2: 0.7980450846662731
n_estimators: 1600 r2: 0.798575365953557
n_estimators: 1700 r2: 0.7984624296743023
n_estimators: 1800 r2: 0.7984765522448803
n_estimators: 1900 r2: 0.798509490780255
n_estimators: 2000 r2: 0.7982179728635121


In [277]:
data = pd.read_csv("d3_3_1418_c.csv", sep=",")
data = data.drop('Healthcare_1', axis=1)
map_AB = {
    'A': 0,
    'B': 1
}
data["Ecology_2"] = data["Ecology_2"].map(map_AB)
data["Ecology_3"] = data["Ecology_3"].map(map_AB)
data["Shops_2"] = data["Shops_2"].map(map_AB)
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())

In [278]:
X = data.drop(['Price', 'Id'], axis = 1)#, 'Rooms', 'LifeSquare'
y = data[['Price']]

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, test_size=0.2, random_state=42)

model_3 = RandomForestRegressor(max_depth=16, n_estimators=1600, random_state=42)
model_3.fit(X_train_3, y_train_3.values[:, 0])
y_pred_3 = model_3.predict(X_test_3)
full_set_err_3 = r2_score(y_test_3, y_pred_3)
full_set_err_3

0.7768932760426173

In [279]:
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()

y_test_all = pd.concat([y_test, y_test_2, y_test_3], axis=0, ignore_index=True)
y_test_pred_all = pd.concat([pd.DataFrame(y_pred, columns=['Price']), pd.DataFrame(y_pred_2, columns=['Price']), pd.DataFrame(y_pred_3, columns=['Price'])], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))

summ r2: 0.748347720507595


### Анализ влияния набора фичей на коэффициент детерминации

In [87]:
def feature_test(X, y, best_r2):
    max_r2 = 0
    max_r2_ind = 0
    new_best_r2 = best_r2
    for ind in range(0, X.shape[1]):
        X_drop = X.drop(X.keys()[ind], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X_drop, y, test_size=0.3, random_state=42)
        model = RandomForestRegressor(max_depth=12, n_estimators=1000, random_state=42)
        model.fit(X_train, y_train.values[:, 0])
        y_pred = model.predict(X_test)
        temp_r2 = r2_score(y_test, y_pred)
        feature_score = temp_r2 - best_r2
        if feature_score > max_r2:
            new_best_r2 = temp_r2
            max_r2 = feature_score
            max_r2_ind = ind
        print("{} {}: {} r2: {}".format((ind + 1), X.keys()[ind], feature_score, temp_r2))
    if max_r2:
        print("drop: {}  r2: {}".format(X.keys()[max_r2_ind], max_r2))
        X.drop(X.keys()[max_r2_ind], axis=1, inplace=True)
        feature_test(X, y, new_best_r2)
    else:
        return new_best_r2


In [88]:
new_r2 = feature_test(X, y, full_set_err)
print(new_r2)

1 DistrictId: 0.00054952526734664 r2: 0.6806194340406364
2 Rooms: -0.0020024230225006434 r2: 0.6780674857507891
3 Square: -0.016803224460555377 r2: 0.6632666843127344
4 LifeSquare: -0.004825663025948668 r2: 0.6752442457473411
5 KitchenSquare: -0.003672376682465339 r2: 0.6763975320908244
6 Floor: -0.006186761371267502 r2: 0.6738831474020223
7 HouseFloor: -0.008228525723642521 r2: 0.6718413830496472
8 HouseYear: -0.008724467624271481 r2: 0.6713454411490183
9 Ecology_1: -0.01155260736910435 r2: 0.6685173014041854
10 Ecology_2: -0.001713444483078863 r2: 0.6783564642902109
11 Ecology_3: -0.0017218068248017193 r2: 0.678348101948488
12 Social_1: -0.004460624325807183 r2: 0.6756092844474826
13 Social_2: -0.003649467719970545 r2: 0.6764204410533192
14 Social_3: -0.03420994553553469 r2: 0.6458599632377551
15 Helthcare_2: -0.0025760721044165713 r2: 0.6774938366688732
16 Shops_1: -0.00167473472532631 r2: 0.6783951740479635
17 Shops_2: -0.0021234472848781216 r2: 0.6779464614884116
drop: DistrictId 

KeyboardInterrupt: 

#### Получен оптимизированный набор фичей

In [250]:
#X = X.drop(['Id', 'LifeSquare'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X.keys()

Index(['DistrictId', 'Square', 'KitchenSquare', 'Floor', 'HouseFloor',
       'HouseYear', 'Ecology_1', 'Ecology_2', 'Social_1', 'Social_2',
       'Social_3', 'Helthcare_2', 'Shops_2'],
      dtype='object')

In [251]:
model = RandomForestRegressor(max_depth=12, n_estimators=1000, random_state=42)
model.fit(X_train, y_train.values[:, 0])
y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.7177086807494449

### Поиск оптимального значения параметра n_estimators

In [102]:
for ind in range(1, 6):
    model = RandomForestRegressor(max_depth=12, n_estimators = ind * 100, random_state=42)
    model.fit(X_train, y_train.values[:, 0])
    y_pred = model.predict(X_test)
    print("n_estimators: {} r2: {}".format(ind * 100, r2_score(y_test, y_pred)))

n_estimators: 100 r2: 0.7236333707278544
n_estimators: 200 r2: 0.722398580435694
n_estimators: 300 r2: 0.7230010205473956
n_estimators: 400 r2: 0.7234748831435995
n_estimators: 500 r2: 0.7240255359287218


### Поиск оптимального значения параметра max_depth

In [103]:
for ind in range(11, 18):
    model = RandomForestRegressor(max_depth = ind, n_estimators = 500, random_state=42)
    model.fit(X_train, y_train.values[:, 0])
    y_pred = model.predict(X_test)
    print("max_depth: {} r2: {}".format(ind, r2_score(y_test, y_pred)))

max_depth: 11 r2: 0.7216817669997949
max_depth: 12 r2: 0.7240255359287218
max_depth: 13 r2: 0.7244070682898032
max_depth: 14 r2: 0.7246481756940918
max_depth: 15 r2: 0.7245751491110739
max_depth: 16 r2: 0.724357545917808
max_depth: 17 r2: 0.7238675675838055


#### Определение коэффициента детерминации с оптимизированными параметрами RandomForestRegressor

In [104]:
model = RandomForestRegressor(max_depth=14, n_estimators=500, random_state=42)
model.fit(X_train, y_train.values[:, 0])
y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.7246481756940918

## Применение KNeighborsRegressor

In [90]:
from sklearn.preprocessing import StandardScaler

X = data.drop(['Id', 'Price'], axis = 1)
y = data[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=6)
model.fit(X_train_scaled, y_train.values[:, 0])
y_pred = model.predict(X_test_scaled)
r2_score(y_test, y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


0.6136556517819667

In [91]:
for ind in range(3, 10):
    model = KNeighborsRegressor(n_neighbors=ind)
    model.fit(X_train_scaled, y_train.values[:, 0])
    y_pred = model.predict(X_test_scaled)
    print(r2_score(y_test, y_pred))

0.5625577107934447
0.5959586388870514
0.6057408716640225
0.6136556517819667
0.6253906381786057
0.6257231296929215
0.6256183319300712


## Применение TSNE и KMeans

In [92]:
from sklearn.preprocessing import StandardScaler

X = data.drop(['Id', 'Price'], axis = 1)
#X = data.drop(['Id', 'LifeSquare', 'Rooms', 'Ecology_3', 'Price'], axis=1)
#X = data[['Rooms', 'Square']]
#X = data[['DistrictId', 'Rooms', 'Square', 'Social_1', 'Helthcare_2', 'LifeSquare', 'KitchenSquare', 'Social_2']]
y = data[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


### TSNE n_components = 2

In [93]:
tsne = TSNE(n_components=2, learning_rate=250, random_state=42)

X_train_tsne = tsne.fit_transform(X_train_scaled)

print('До:\t{}'.format(X_train_scaled.shape))
print('После:\t{}'.format(X_train_tsne.shape))

До:	(4043, 17)
После:	(4043, 2)


In [94]:
#plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1])

#plt.show()

In [95]:
kmeans = KMeans(n_clusters=20, random_state=42)

labels_train = kmeans.fit_predict(X_train_scaled)

#plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=labels_train)

#plt.show()

In [96]:
labels_test = kmeans.predict(X_test_scaled)

In [97]:
lr = LinearRegression()

lr.fit(X_train_scaled, y_train)

y_test_pred = lr.predict(X_test_scaled)

r2_score(y_test, y_test_pred)

0.567000646615303

In [98]:
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()

for ind in range(0, 20):
    lr.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
    y_test_pred = lr.predict(X_test_scaled.loc[labels_test == ind])
    print(r2_score(y_test[labels_test == ind], y_test_pred))
    y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))

0.2617356297293002
0.4929475338857561
0.26880641732437094
0.8239626644742697
0.41398993174130283
-0.13074875948820597
0.3190659097823011
0.1337510068408294
0.38784643646847505
0.553124379436687
0.42235058703285633
-1.5434801449825293
0.45975868354444027
0.5313572547127731
0.5741349993588958
-0.20819096064134057
0.0
0.5218449428454164
0.4052779407055209
0.4622725227114286
summ r2: 0.6081747569775159


In [99]:
for ind_2 in range(3, 30):
    kmeans = KMeans(n_clusters=ind_2, random_state=42)
    labels_train = kmeans.fit_predict(X_train_scaled)
    labels_test = kmeans.predict(X_test_scaled)

    
    y_test_all = pd.DataFrame()
    y_test_pred_all = pd.DataFrame()

    for ind in range(0, ind_2):
        lr.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
        y_test_pred = lr.predict(X_test_scaled.loc[labels_test == ind])
        #print(r2_score(y_test[labels_test == ind], y_test_pred))
        y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
        y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
    print("ind: {} summ r2: {}".format(ind_2, r2_score(y_test_all, y_test_pred_all)))

ind: 3 summ r2: 0.5792885432990238
ind: 4 summ r2: 0.5758736725759543
ind: 5 summ r2: 0.584406097795485
ind: 6 summ r2: 0.5916388851602778
ind: 7 summ r2: 0.6020261581845792
ind: 8 summ r2: 0.6042307368348312
ind: 9 summ r2: 0.6007223245499803
ind: 10 summ r2: 0.5948824208281192
ind: 11 summ r2: 0.59057935999914
ind: 12 summ r2: 0.5930101374062069
ind: 13 summ r2: 0.5939187633532146
ind: 14 summ r2: 0.5989304911344555
ind: 15 summ r2: 0.6092137080984922
ind: 16 summ r2: 0.5984030963199327
ind: 17 summ r2: 0.5713928755943211
ind: 18 summ r2: 0.6102101781947122
ind: 19 summ r2: 0.6053749046885426
ind: 20 summ r2: 0.6081747569775159
ind: 21 summ r2: 0.629708257968564
ind: 22 summ r2: 0.5789594141650005
ind: 23 summ r2: 0.5917486208830367
ind: 24 summ r2: 0.6226970779294547
ind: 25 summ r2: 0.6148416818574773
ind: 26 summ r2: 0.617180624992458
ind: 27 summ r2: 0.6056901912444383
ind: 28 summ r2: 0.6136475634371974
ind: 29 summ r2: 0.6122720647448402


In [100]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=6)
model.fit(X_train_scaled, y_train.values[:, 0])
y_pred = model.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.6136556517819667

In [101]:
for ind in range(3, 11):
    model = KNeighborsRegressor(n_neighbors=ind)
    model.fit(X_train_scaled, y_train.values[:, 0])
    y_pred = model.predict(X_test_scaled)
    print(r2_score(y_test, y_pred))

0.5625577107934447
0.5959586388870514
0.6057408716640225
0.6136556517819667
0.6253906381786057
0.6257231296929215
0.6256183319300712
0.6286555353052556


In [103]:
kmeans = KMeans(n_clusters=20, random_state=42)
labels_train = kmeans.fit_predict(X_train_scaled)
labels_test = kmeans.predict(X_test_scaled)

y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()
model = KNeighborsRegressor(n_neighbors=9)
for ind in range(0, 20):
    model.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
    y_test_pred = model.predict(X_test_scaled.loc[labels_test == ind])
    print(r2_score(y_test[labels_test == ind], y_test_pred))
    y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))  

0.20740032529342411
0.40960764136565064
0.32252136008237486
0.7490202893969622
0.4372104202441748
0.3139812188495069
0.4904457767249385
0.17883969353138274
0.4007653683888174
0.42384426193521907
0.41814185741498333
0.048600748580908903
0.41545008131114514
0.47791639141277575
0.5234538804715436
0.3045728875313318


ValueError: Expected n_neighbors <= n_samples,  but n_samples = 5, n_neighbors = 9

In [104]:
model = RandomForestRegressor(max_depth=14, n_estimators=500, random_state=42)
model.fit(X_train_scaled, y_train.values[:, 0])
y_pred = model.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.6791956399045251

In [105]:
kmeans = KMeans(n_clusters=20, random_state=42)
labels_train = kmeans.fit_predict(X_train_scaled)
labels_test = kmeans.predict(X_test_scaled)

y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()
model = RandomForestRegressor(max_depth=14, n_estimators=500, random_state=42)
for ind in range(0, 20):
    model.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
    y_test_pred = model.predict(X_test_scaled.loc[labels_test == ind])
    print(r2_score(y_test[labels_test == ind], y_test_pred))
    y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))  

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.16708378413194147


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.41563946593888657


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


...

0.5285638246906701


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.31906683573928174


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.4942034120224372
summ r2: 0.6371128654332461


### TSNE n_components = 3

In [129]:
from sklearn.preprocessing import StandardScaler

X = data.drop(['Id', 'Price'], axis = 1)
#X = data.drop(['Id', 'LifeSquare',  'Ecology_3', 'Price'], axis=1)  #'Rooms',
#X = data[['Rooms', 'Square']]
#X = data[['DistrictId', 'Rooms', 'Square', 'Social_1', 'Helthcare_2', 'LifeSquare', 'KitchenSquare', 'Social_2']]
y = data[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [130]:
tsne = TSNE(n_components=3, learning_rate=250, random_state=42)

X_train_tsne = tsne.fit_transform(X_train_scaled)

print('До:\t{}'.format(X_train_scaled.shape))
print('После:\t{}'.format(X_train_tsne.shape))

До:	(6981, 17)
После:	(6981, 3)


In [77]:
#from mpl_toolkits.mplot3d import Axes3D
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], X_train_tsne[:, 2])

#plt.show()

In [78]:
kmeans = KMeans(n_clusters=20, random_state=42)

labels_train = kmeans.fit_predict(X_train_scaled)

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], X_train_tsne[:, 2], c=labels_train)

#plt.show()

In [133]:
labels_test = kmeans.predict(X_test_scaled)

In [134]:
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()

lr = LinearRegression()

for ind in range(0, 20):
    lr.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
    y_test_pred = lr.predict(X_test_scaled.loc[labels_test == ind])
    print(r2_score(y_test[labels_test == ind], y_test_pred))
    y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))

0.7097813562052143
0.5465078288064105
0.05001039335075197
0.11787693903245255
0.5708446899557618
0.6627937734305158
0.472621018625097
0.5342964511400548
0.4396248251136795
0.6834711518566868
0.40939633947302156
0.3914573423349047
0.5944280663547953
0.3463748738696484
0.5275819062685667
0.7266161261011728
0.8166543043393804
0.36436632166333494
0.6527336971919802
0.3227438102892225
summ r2: 0.6547804907725527


In [135]:
for ind_2 in range(3, 30):
    kmeans = KMeans(n_clusters=ind_2, random_state=42)
    labels_train = kmeans.fit_predict(X_train_scaled)
    labels_test = kmeans.predict(X_test_scaled)

    
    y_test_all = pd.DataFrame()
    y_test_pred_all = pd.DataFrame()

    for ind in range(0, ind_2):
        lr.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
        y_test_pred = lr.predict(X_test_scaled.loc[labels_test == ind])
        #print(r2_score(y_test[labels_test == ind], y_test_pred))
        y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
        y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
    print("ind: {} summ r2: {}".format(ind_2, r2_score(y_test_all, y_test_pred_all)))

ind: 3 summ r2: 0.5958724717937405
ind: 4 summ r2: 0.6071608477138961
ind: 5 summ r2: 0.6080645631900292
ind: 6 summ r2: 0.6058040102505289
ind: 7 summ r2: 0.6058012043302463
ind: 8 summ r2: 0.6161353502713008
ind: 9 summ r2: 0.6154647759067754
ind: 10 summ r2: 0.6213562908553601
ind: 11 summ r2: 0.6161736981664132
ind: 12 summ r2: 0.6226933182207542
ind: 13 summ r2: 0.6249111108367915
ind: 14 summ r2: 0.6311787520045049
ind: 15 summ r2: 0.6455742850335853
ind: 16 summ r2: 0.6495480142797995
ind: 17 summ r2: 0.6394221043104067
ind: 18 summ r2: 0.6517429119948606
ind: 19 summ r2: 0.6409369179152147
ind: 20 summ r2: 0.6547804907725527
ind: 21 summ r2: 0.6674065494994674
ind: 22 summ r2: 0.6421404092006533
ind: 23 summ r2: 0.6668826751127421
ind: 24 summ r2: 0.6453692990126447
ind: 25 summ r2: 0.6428121500232671
ind: 26 summ r2: 0.6516760628419891
ind: 27 summ r2: 0.6495893518668714
ind: 28 summ r2: 0.6135951147048504
ind: 29 summ r2: 0.637550037610668


In [136]:
kmeans = KMeans(n_clusters=20, random_state=42)
labels_train = kmeans.fit_predict(X_train_scaled)
labels_test = kmeans.predict(X_test_scaled)

y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()
model = KNeighborsRegressor(n_neighbors=9)
for ind in range(0, 20):
    model.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
    y_test_pred = model.predict(X_test_scaled.loc[labels_test == ind])
    print(r2_score(y_test[labels_test == ind], y_test_pred))
    y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))   

0.7300566737044226
0.4740917998907921
-0.06459193841381294
0.2813894593541413
0.6026526162413061
0.629583031176119
0.40077082347484405
0.5052056705197556
0.3460423240649855
0.5509215924159532
0.34259744185327734
0.31769248654495075
0.4752132609999773
0.4827165966426975
0.4078305427454799
0.6764955760891888
0.3964375465487171
0.2153934187619415
0.4242596381538879
0.2800725539409814
summ r2: 0.619378706213032


In [137]:
kmeans = KMeans(n_clusters=20, random_state=42)
labels_train = kmeans.fit_predict(X_train_scaled)
labels_test = kmeans.predict(X_test_scaled)

y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()
model = RandomForestRegressor(max_depth=16, n_estimators=500, random_state=42)
for ind in range(0, 20):
    model.fit(X_train_scaled.loc[labels_train == ind], y_train[labels_train == ind])
    y_test_pred = model.predict(X_test_scaled.loc[labels_test == ind])
    print(r2_score(y_test[labels_test == ind], y_test_pred))
    y_test_all = pd.concat([y_test_all, y_test[labels_test == ind]], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_test_pred)], axis=0, ignore_index=True)
print("summ r2: {}".format(r2_score(y_test_all, y_test_pred_all)))    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.7098040701915409


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.6957834675543253


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


...

0.3360297776824561


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.640731092783017


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.6749172240529999
summ r2: 0.7063190031536403


### Максимальное значение коэффициента детерминации: выделение по признакам трёх диапазонов и построении отдельных моделей RandomForestRegressor (для первой группы используется модель по всему датасету)

In [124]:
data = pd.read_csv("train_cor.csv", sep=",")
data_test = pd.read_csv("test_cor.csv", sep=",")

In [125]:
X = data.drop(['Id', 'Price', 'LifeSquare'], axis=1)
y = data[['Price']]
res = data_test[['Id']].copy()
data_test = data_test.drop(['Id', 'LifeSquare'], axis=1)

In [126]:
model = RandomForestRegressor(max_depth=16, n_estimators=700, random_state=42)
model.fit(X, y.values[:, 0])
y_pred = model.predict(data_test)
y_pred

array([162290.0590456 , 223580.65823862, 340652.8334451 , ...,
       335652.16682194, 207461.71778972, 177736.1753492 ])

In [127]:
res = pd.concat([res, pd.DataFrame(y_pred, columns=['Price'])], axis=1)

In [128]:
res.to_csv("VGladyshev_predictions.csv", sep=",", index=False)